# Context Dependent Generalization
____
In this demonstration, we show that it is often beneficial to genearalize *skills* seperately from *goals*. In the following, simple artificial agents learn both how to navigate (a skill) and to find reward (a goal) across several highly similar tasks. Each task has a context, and if the agents use the context to generalize, they will see a performance benefit when the contexts are similar.

We present three agents. One agent (the "Flat" agent) does not generalize between context. A second (the "Joint Clustering" agent), clusters contexts as a whole, assuming that each member of a context cluster has the *exact* same statistics. The final (the "Independent Clustering" agent), clusters skills and goals seperately, assuming a paritial similiarity across contexts. 


In [1]:
%matplotlib inline
import copy
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm_notebook

from Model.Agents import JointClustering, IndependentClusterAgent, FlatAgent
from Model.GridWorld import make_task

# Define a function to Simluate the Models
def simulate(agent_class, simulation_number, task_kwargs, agent_kwargs=None):
    _kwargs = copy.copy(task_kwargs)
    del _kwargs['list_goal_priors']
    task = make_task(**_kwargs)
    if agent_kwargs is not None:
        agent = agent_class(task, **agent_kwargs)
    else:
        agent = agent_class(task)
    agent.set_reward_prior(task_kwargs['list_goal_priors'])

    results = agent.generate()
    results['Simulation Number'] = [simulation_number] * len(results)
    results['Cumulative Steps Taken'] = results['n actions taken'].cumsum()

    return results

def simulate_all(n_sim, task_kwargs, agent_kwargs=None, alpha=2.0):
    if agent_kwargs is None:
        agent_kwargs=dict(alpha=alpha)
    elif 'alpha' not in agent_kwargs.keys():
        agent_kwargs['alpha'] = alpha
        
    resultsJC = [None] * n_sim
    resultsIC = [None] * n_sim
    resultsFL = [None] * n_sim
    for ii in tqdm_notebook(range(n_sim)):
        resultsJC[ii] = simulate(JointClustering, ii, task_kwargs, agent_kwargs)
        resultsIC[ii] = simulate(IndependentClusterAgent, ii, task_kwargs, agent_kwargs)
        resultsFL[ii] = simulate(FlatAgent, ii, task_kwargs)

    resultsJC = pd.concat(resultsJC)
    resultsIC = pd.concat(resultsIC)
    resultsFL = pd.concat(resultsFL)
    
    resultsJC['Model'] = ['Joint Clusterting'] * len(resultsJC)
    resultsIC['Model'] = ['Independent Clustering'] * len(resultsIC)
    resultsFL['Model'] = ['Flat Agent'] * len(resultsFL)
    return pd.concat([resultsJC, resultsIC, resultsFL])

### Description of the Agents
______

This Demonstration compares performance of reinforcement learning models across seperate seperate tasks. Each task consists of a series of trials with a single "goal" (a self-absorbing state with positive reward) hidden in a grid world. An agent is randomly place in the grid at the start of a trial and has to reach the goal before proceeding to the next trial.

* Each trial is associated with a *context* and the state of the agent can be defined formally as a tuple of its grid-coordinates and context:  
$$ s = (x, y, c)$$

* As this task models human behavior, we assume the agent knows the simple structure of the maze. We assume the agent knows cardinal directions, that is, "north" in one square is equivalent to "north" in another square. To navigate the maze, therefore, the agent has to learn the relationship between the primitive actions it can take and its own subsequent caridinal movements. There are eight primitive actions learning this relationship between actions and movements is a simple, generalizable "skill" that can potententially be transfered to new contexts efficaciously. Formally, these skills are a function that takes in an action and outputs a cardinal movement: 
$$ \phi: a \rightarrow m 
$$

* Independently of skill aquisition, the agents also learn a representation of goals in the form of a reward function, which is a mapping between states $s$ and rewards:
$$
R: s \rightarrow r $$

* The agents integrate over their skills and goals to plan. This is done with dynamic programming (policy iteration).
\br

The agents are different only in how they utalize the context:
* Flat Agent: Assumes each context is independent
* Joint Clustering Agent: Assumes contexts can be clustered together and clusters share both reward function and mapping
* Independent Clustering Agent: Assumes reward functions and mappings can be clustered indpendendly of each other based on their contexts

Cluster assignements are inferred with Bayesian inference. We assume all members of a cluster have the same statistics. For the joint model, this is both the skill and reward functions, while for the independent model it is either. These statistics, the learned skill and reward functions constitute the likelihood for Bayesian infernece.

For the prior, we assume a popularity-based generative process that lets the number of clusters grow with new contexts but prefers a small number of clusters. The degree to which the agents cluster depends on the parameter $\alpha$ that allows us to control the degree to which models cluster. Low alpha values corresponding to an increased tendency to cluster. High alpha values lead to less clustering and as alpha approaches infinity, both clustering collapse to the Flat model.

Formally, the prior over contexts is defined with a CRP where the probability that context *c* is in cluster *k* is defined:
$$\Pr\left (c_{t+1} = k| c_{1:t} \right ) = 
\begin{cases} 
N_k/A & \quad \text{if } k < K_t \\
\alpha/A & \quad \text{if } k_K+1 \\
\end{cases}
$$

Where $N_k$ is the number of contexts associated with cluster *k* and $A=\sum_{k}N_k$ is a normalizing constant. As a consequence of this prior, clusters that have more associated contexts will have higher prior probability over new contexts.


# Task 1
The first set of simulations are on a set of 8x8 gridworlds. There are six contexts with
a total of 4 goal locations (one at each corner) and 2 skill sets (relationship between actions and movements). No two contexts share both a goal and a skill set, but generalization is still possible between them.

In [2]:
# define all of the task parameters
grid_world_size = (6, 6)

# define mapping between primitive action (a in [0, 1]) and cardinal movements
# (left, right, up down)
map_0 = {0: u'left', 1: u'up', 2: u'down', 3: u'right'}
map_1 = {4: u'up', 5: u'left', 6: u'right', 7: u'down'}

# define the mappings for each context, where the ith mapping belongs the ith context
context_maps = [map_0, map_0, map_0, map_0, map_1, map_1]

# define goal locations 
goal_0 = (0, 0)
goal_1 = (0, 5)
goal_2 = (5, 0)
goal_3 = (5, 5)

# define the goal location for each context, where the ith goal belongs the ith context
context_goals = [goal_0, goal_1, goal_2, goal_3, goal_0, goal_3]

# randomly start the agent somewhere in the middle of the map
start_locations = [(x, y) for x in range(1, 5) for y in range(1, 5)]

# the number of times each context is shown
context_balance = [2, 2, 6, 6, 4, 4]

# the hazard rate determines a degree of auto correlation in the context orders. This is
# useful in human studies. The hazard rates is the defined by the probability of a 
# context change after i repeats is f(i)
hazard_rates = [0.5, 0.67, 0.67, 0.75, 1.0, 1.0]

# we include a prior in the reward function over the goal locations
list_goal_priors = [(0, 0), (0, 5), (5, 0), (5, 5)]

task_kwargs = dict(context_balance=context_balance, context_goals=context_goals, 
                   context_maps=context_maps, hazard_rates=hazard_rates,
                   start_locations=start_locations, grid_world_size=grid_world_size,
                   list_goal_priors=list_goal_priors)

Now, we simulate each model 500 times (this can take a while) and plot the cumulative number of steps taken across 
trials. In the task, the agent must reach the goal (an absorbing state with positive reward)
to advance to the next trial. Lower cumulative steps taken indicated faster learning.

In [ ]:
n_sim = 20

results = simulate_all(n_sim, task_kwargs)

# define the parameters to plot the results
tsplot_kwargs = dict(
    time='TrialNumber',
    value='Cumulative Steps Taken',
    data=results[results['In goal']], 
    unit='Simulation Number',
    condition='Model',
    estimator=np.median
)

with sns.axes_style('ticks'):
    sns.tsplot(**tsplot_kwargs)
    sns.despine()

We can see in this demonstration the agent that clusters reward functions and mappings between actions and movements independently is meaningfully faster that either agents that jointly cluster or agents that do not cluster.

## Task 2: Smaller Room
In the example above, jointly clustering goals and mappings ends up being faster than no clustering at all because there is still an advantagege of clustering the mappings. We can see this by shrinking the gridworld down to a 3x3 maze where the agent starts one step from all 4 goal locations, minimizing the benefit of mapping generalization

In [ ]:
# define all of the task parameters here
grid_world_size = (3, 3)

goal_0 = (0, 1)
goal_1 = (1, 0)
goal_2 = (2, 1)
goal_3 = (2, 2)
context_goals = [goal_0, goal_1, goal_2, goal_3, goal_0, goal_3]

# randomly start the agent somewhere in the middle of the map
start_locations = [(1, 1)]

list_goal_priors = [goal_0, goal_1, goal_2, goal_3]

task_kwargs = dict(context_balance=context_balance, context_goals=context_goals, 
                   context_maps=context_maps, hazard_rates=hazard_rates,
                   start_locations=start_locations, grid_world_size=grid_world_size,
                   list_goal_priors=list_goal_priors)

# simulate the agents
results = simulate_all(n_sim, task_kwargs)

tsplot_kwargs['data'] = results[results['In goal']]
with sns.axes_style('ticks'):
    sns.tsplot(**tsplot_kwargs)
    sns.despine()

We can see that independent clustering results still results in a generalization benefit where as joint clustering has a meaningful generalization cost. It's also worse noting that both models see a penalty early on as compared to a model that does not cluster. This early cost of generalization is typical of generalization learning is made up overtime as the agent encounters new enviornments.

## Task 3: Joint Clustering
Here, we show a task where it is useful to generalize contexts as a whole. Each context that shares a reward function with another context also shares its mapping between actions and movements. As such, generalizing the context as a whole is useful.

In [ ]:
# define all of the task parameters
grid_world_size = (6, 6)

# define the mappings for each context, where the ith mapping belongs the ith context
context_maps = [map_0, map_0, map_0, map_1, map_1, map_1]

# define goal locations 
goal_0 = (0, 0)
goal_1 = (0, 5)
goal_2 = (5, 0)
goal_3 = (5, 5)
# define the goal location for each context, where the ith goal belongs the ith context
context_goals = [goal_0, goal_0, goal_0, goal_3, goal_3, goal_3]

start_locations = [(x, y) for x in range(1, 5) for y in range(1, 5)]

context_balance = [4, 4, 4, 4, 4, 4]

list_goal_priors = [(0, 0), (0, 5), (5, 0), (5, 5)]

task_kwargs = dict(context_balance=context_balance, context_goals=context_goals, 
                   context_maps=context_maps, hazard_rates=hazard_rates,
                   start_locations=start_locations, grid_world_size=grid_world_size,
                   list_goal_priors=list_goal_priors)

results = simulate_all(n_sim, task_kwargs, alpha=0.5)

tsplot_kwargs['data'] = results[results['In goal']]
with sns.axes_style('ticks'):
    sns.tsplot(**tsplot_kwargs)
    sns.despine()

Here, we can see that clustering the reward functions and mappings together outperforms the flat agent. We can also construct enviornments in which there is a substantial benefit to this type of generalization. However, independently clustering provides a more flexable method that does not suffer the same performance penalties as the joint clustering. 

Notice that this also depends on parameterization. Here, it useful for have a lower value of alpha as the number of useful clusters is much lower than the number of contexts.